In [1]:
!pip3.8 install setuptools
!python3.8 -m pip install packaging
!python3.8 -m pip install pandas dash
!pip install more-itertools

zsh:1: command not found: pip3.8
zsh:1: command not found: python3.8
zsh:1: command not found: python3.8


In [3]:
!pip install dash
!pip install plotly

In [7]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Create the Dash app
app = dash.Dash(__name__)

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the automobiles data
auto_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Create the layout
app.layout = html.Div([
    # TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}),

    # TASK 2.2: Add two dropdown menus
    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            placeholder='Select a report type',
            style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        ),
        
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(year), 'value': year} for year in range(1980, 2024)],
            placeholder='Select year',
            style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        )
    ]),

    # TASK 2.3: Add a division for output display
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

# TASK 2.4 Creating Callbacks
# Callback to enable/disable year dropdown
@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    return True

# Callback for updating output container
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'),
     Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # TASK 2.5: Add code for recession statistics
        recession_data = auto_data[auto_data['Recession'] == 1]
        
        # Plot 1: Sales fluctuation during recession
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                          x='Year',
                          y='Automobile_Sales',
                          title='Average Automobile Sales Fluctuation Over Recession Period')
        )
        
        # Plot 2: Average number of vehicles sold by vehicle type
        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(avg_sales,
                         x='Vehicle_Type',
                         y='Automobile_Sales',
                         title='Average Number of Vehicles Sold by Vehicle Type During Recessions')
        )
        
        # Plot 3: Total expenditure share by vehicle type
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
                         values='Advertising_Expenditure',
                         names='Vehicle_Type',
                         title='Total Expenditure Share by Vehicle Type During Recessions')
        )
        
        # Plot 4: Effect of unemployment rate on vehicle type and sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data,
                         x='unemployment_rate',
                         y='Automobile_Sales',
                         color='Vehicle_Type',
                         title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )

        return [
            html.Div(children=[R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div(children=[R_chart3, R_chart4], style={'display': 'flex'})
        ]

    elif selected_statistics == 'Yearly Statistics' and input_year:
        # TASK 2.6: Add code for yearly statistics
        yearly_data = auto_data[auto_data['Year'] == input_year]
        
        # Plot 1: Yearly automobile sales
        yas = auto_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas,
                          x='Year',
                          y='Automobile_Sales',
                          title='Yearly Automobile Sales')
        )
        
        # Plot 2: Monthly automobile sales for specific year
        month_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(month_sales,
                          x='Month',
                          y='Automobile_Sales',
                          title=f'Monthly Automobile Sales in {input_year}')
        )
        
        # Plot 3: Average vehicles sold by type
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata,
                         x='Vehicle_Type',
                         y='Automobile_Sales',
                         title=f'Average Vehicles Sold by Vehicle Type in {input_year}')
        )
        
        # Plot 4: Total advertisement expenditure by vehicle
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
                         values='Advertising_Expenditure',
                         names='Vehicle_Type',
                         title=f'Total Advertisement Expenditure for Each Vehicle in {input_year}')
        )
        
        return [
            html.Div(children=[Y_chart1, Y_chart2], style={'display': 'flex'}),
            html.Div(children=[Y_chart3, Y_chart4], style={'display': 'flex'})
        ]

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)